In [21]:
import re
import requests
from bs4 import BeautifulSoup

MOBILE_UA = (
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) "
    "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 "
    "Mobile/15E148 Safari/604.1"
)

# URL = "https://www.letras.mus.br/the-beatles/182/"
# URL = "https://www.letras.mus.br/the-beatles/95/"
URL = "https://www.letras.mus.br/lacrimosa/22112/"

def clean(text: str) -> str:
    text = text.replace("\r", "")
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def scrape_letras(url: str):
    headers = {
        "User-Agent": MOBILE_UA,
        "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    }
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    title_el = soup.select_one("h1.textStyle-primary")
    artist_el = soup.select_one("h2.textStyle-secondary")

    # letra original costuma estar aqui (como no seu HTML)
    left = soup.select_one(".lyric-translation-left")
    if not left:
        # fallback (caso a estrutura mude)
        left = soup.select_one(".lyric-content")

    lyrics_parts = []
    if left:
        for p in left.find_all("p"):
            chunk = clean(p.get_text("\n", strip=True))
            if chunk:
                lyrics_parts.append(chunk)

    lyrics = clean("\n\n".join(lyrics_parts))

    return {
        "title": title_el.get_text(strip=True) if title_el else None,
        "artist": artist_el.get_text(strip=True) if artist_el else None,
        "lyrics": lyrics,
    }

if __name__ == "__main__":
    data = scrape_letras(URL)
    print("Música:", data["title"])
    print("Artista:", data["artist"])
    print("\nLetra:\n")
    print(data["lyrics"])

Música: Alleine Zu Zweit
Artista: Lacrimosa

Letra:

Am Ende der Wahrheit, am Ende des Lichts
Am Ende der Liebe, am Ende, da stehst du
Im Herzen wird es leerer, ein Teil geht nun von mir
Nichts hat überlebt, wir haben schweigend uns schon lange getrennt
Und mit jedem Tag wir wuchs die Lüge unserer Liebe
Und je weiter wir den Weg zusammen gingen
Desto weiter haben wir uns voneinander entfernt

Einsam, gemeinsam, wir haben verlernt uns neu zu suchen
Die Gewohnheit vernebelt, die Trägheit erstickt
Der Hochmut macht trunken, und die Nähe treibt zur Flucht

Tanz, mein Leben, tanz. Tanz mit mir, tanz mit mir
Noch einmal in den puren Rausch der nackten Liebe
Tanz, mein Leben, tanz. Tanz mit mir, tanz mit mir
Noch einmal in den puren Rausch der nackten Liebe

Und wenn ich ihn so sehe, wenn ich sie erlebe
Wenn ich uns betrachte, etwas hat überlebt
Und wenn ich Kraft und Hoffnung fände
Wenn ich selbst noch den Glauben an uns hätte
Wenn ich sie erreichen könnte
Sie noch einmal für mich hätte
Wenn